In [15]:
import pandas as pd
import json

In [82]:
df = pd.read_csv("jobs_with_requirements.csv")
df = df.drop(df.columns[0], axis=1)


In [83]:
#change the type to dictionary to make manipulation easier
df["requirements_extracted"] = df.requirements_extracted.apply(eval)

In [84]:
df["hard_skills"] = df.requirements_extracted.apply(lambda x : x.get("hard_skills"))

In [85]:
df["languages"] = df.requirements_extracted.apply(lambda x: x.get("languages"))

In [86]:
df.requirements_extracted[2]

{'hard_skills': {'programming language': 'other',
  'data science': 'data science & ai'},
 'languages': ['german'],
 'experience': None,
 'education': {'degree': 'degree', 'computer science': 'field'},
 'soft_skills': ['analytical', 'communication', 'initiative']}

In [87]:
df["experience"] = df.requirements_extracted.apply(lambda x: x.get("experience"))

In [88]:
df["degree"] = df.requirements_extracted.apply(lambda x : x.get("education"))

In [89]:
df["soft_skills"] = df.requirements_extracted.apply(lambda x: x.get("soft_skills"))

In [90]:
df = df.drop(columns= ["requirements_extracted"],axis =1)

In [91]:
df["skills"] = df.hard_skills.apply(lambda x : list(x.keys()))

In [92]:
df[df["skills"].apply(lambda x : len(x)) == 0]

,title,company,location,job_type,employment_type,description,requirements,job_category,hard_skills,languages,experience,degree,soft_skills,skills
27,- Data Science Artificial Intelligence Autumn,ZEISS,Oberkochen (Baden-Württemberg),"Ausbildung, Studium",Vollzeit,"Be the change i be sgshißen for curious, passi...",You have completed studies with technical-scie...,IT|Data & AI|Data Science,{},None,None,None,"[management, communication]",[]
29,- Data Science Artificial Intelligence,Schaeffler Technologies AG & Co. KG,Herzogenaurach,"Ausbildung, Studium",Vollzeit,We are the Federal Agency for Cartography and ...,Study course in the field of industrial engine...,IT|Data & AI|Data Science,{},"[german, english]",None,"{'degree': 'degree', 'economic': 'field', 'eng...",None,[]
34,Full-Stack Data Analytics Data Science Expert ...,Mercedes-Benz AG,Stuttgart,Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit",Diedrescher Consulting GmbHist ANDER -RIGHTIC ...,Several years of experience as a project manag...,IT|Data & AI|Data Science,{},"[german, english]",None,None,[communication],[]
40,Mathematician Actuar Data Science Pricing Risk...,Meyerthole Siems Kohlruss Ges. für aktuarielle...,Köln,"Berufseinstieg/Trainee, Feste Anstellung",Vollzeit,"Are you an analyst, designer and do you want t...",PhD in life or data sciences and/or MD in medi...,IT|Data & AI|Data Science,{},None,3 years,None,"[collaboration, initiative]",[]
54,Data Scientist,monari GmbH,Gronau,Feste Anstellung,"Homeoffice möglich, Vollzeit",Our passion for technical challenges drives us...,"General university entrance qualifications, su...",IT|Data & AI|Data Science,{},[english],None,None,"[communication, initiative]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3016,Planning Coordinator - Area Data Centers,Ed. Züblin AG,"Frankfurt am Main, Rhein-Main-Gebiet",Feste Anstellung,Vollzeit,As part of Strabag SE and number one in German...,"You have a degree in business administration, ...",IT|Data & AI|Data Analysis,{},"[german, english]",None,"{'degree': 'degree', 'engineering': 'field', '...","[analytical, management]",[]
3017,System Engineer It - Firewall Network Security,Landkreis Emmendingen,Emmendingen,Feste Anstellung,"Homeoffice möglich, Vollzeit",Your attractive employer in the region for str...,You have completed technical training or a deg...,Engineering|Industrial & Systems,{},"[german, english]",None,"{'degree': 'degree', 'engineering': 'field', '...",None,[]
3018,Information Security Advisor,neoshare AG,"München, Frankfurt a. M.",Feste Anstellung,"Homeoffice möglich, Vollzeit","NEOSHARE AG, founded in Munich in 2019, quickl...",General/subject-bound high school graduation s...,Legal|Legal Practice,{},None,None,None,None,[]
3021,Cloud Architekt,ZDF,Mainz,Feste Anstellung,"Teilzeit, Vollzeit",The Planning and Realization business area (PU...,You have a completed training as a bank clerk ...,IT|DevOps & Cloud|Cloud Engineering,{},None,None,None,[communication],[]


## too many rows without skills we need to fix this 

In [110]:
SKILL_CATEGORIES = {
    # --- Programming Languages ---
    'Programming Languages': [
        'python', 'java', 'javascript', 'typescript', 'c++', 'c#', 'go', 'golang',
        'ruby', 'php', 'swift', 'kotlin', 'rust', 'scala', 'r', 'dart', 'perl',
        'bash', 'html', 'css', 'Python', 'oop', 'object-oriented','object-oriented programming'
    ],

    # --- Web Development ---
    'Web Development': [
        'react', 'angular', 'vue', 'svelte', 'next.js', 'nuxt.js', 'jquery','full-stack'
        'd3.js', 'redux', 'webpack', 'babel', 'tailwind', 'bootstrap', 'sass','full stack'
        'less', 'graphql', 'rest', 'api','web development', 'web dev','web developer'
    ],

    # --- Backend Development ---
    'Backend Development': [
        'node.js', 'express', 'django', 'flask', 'spring', 'spring boot',
        'laravel', 'ruby on rails', 'asp.net', '.net core', 'fastapi', 'nestjs', 'full stack',
        'hibernate', 'jpa', 'gin', 'echo','api', 'rest','rest api', 'restful','full-stack'
    ],

    # --- Mobile & Game Development ---
    'Mobile & Game Dev': [
        'react native', 'flutter', 'xamarin', 'ionic', 'unity', 'unreal engine',
        'cocos2d', 'godot', 'opengl', 'directx', 'phaser', 'pixi.js'
    ],

    # --- Cloud & DevOps ---
    'Cloud & DevOps': [
        'aws', 'azure', 'google cloud', 'gcp', 'docker', 'kubernetes',
        'terraform', 'ansible', 'jenkins', 'github actions', 'gitlab ci',
        'circleci', 'argo cd', 'helm', 'prometheus', 'grafana', 'splunk',
        'new relic', 'datadog', 'istio', 'linkerd', 'cloud','cloud computing'
    ],

    # --- Databases ---
    'Databases': [
        'sql', 'postgresql', 'mysql', 'mongodb', 'redis', 'cassandra', 'databases',
        'elasticsearch', 'dynamodb', 'firebase', 'snowflake', 'bigquery','relational',
        'oracle', 'sql server', 'mariadb', 'neo4j', 'cosmos db', 'database','non relational'
        'no sql','nosql','non-relational'
    ],

    # --- Data Science & AI ---
    'Data Science & AI': [
        'pandas', 'numpy', 'scikit-learn', 'tensorflow', 'pytorch', 'keras',
        'opencv', 'spark', 'hadoop', 'hive', 'kafka', 'airflow', 'mlflow',
        'ray', 'nltk', 'spacy', 'hugging face', 'langchain', 'llama', 'gpt',
        'bert', 'transformers', 'computer vision', 'nlp', 'ai', 'Artificial Intelligence',
        'reinforcement learning', 'supervised learning', 'unsupervised learning',
        'ai management', 'ai developement', 'llm', 'deep learning', 'tensorflow',
        'pytorch','data science','data scientist', 'ai engineer', 'ai engineering'
    ],

    # --- Data Engineering ---
    'Data Engineering': [
        'etl', 'elt', 'data pipeline', 'data warehouse', 'data lake',
        'delta lake', 'apache beam', 'apache flink', 'dbt', 'prefect',
        'dagster', 'redshift', 'databricks', 'data strategy', 'data architecture',
        'data foundation', 'data management', 'data engineering', 'data engineer'
    ],

    # --- BI & Analytics ---
    'BI & Analytics': [
        'tableau', 'power bi', 'looker', 'qlikview', 'qliksense', 'bi tools','bi'
        'microstrategy', 'sisense', 'domo', 'matplotlib', 'seaborn', 'dashboards',
        'plotly', 'ggplot', 'excel', 'google sheets', 'vba', 'pbi','dashboard',
        'business intelligence','visualization', 'data analysis','data visualization'
    ],

    # --- Cybersecurity ---
    'Cybersecurity': [
        'owasp', 'penetration testing', 'metasploit', 'burp suite',
        'wireshark', 'nmap', 'siem', 'soc', 'ids', 'ips', 'firewall',
        'vpn', 'zero trust', 'saml', 'oauth', 'openid', 'iso 27001',
        'gdpr', 'hipaa', 'pci dss', 'nist', 'cis controls','security'
    ],

    # --- QA & Testing ---
    'QA & Testing': [
        'selenium', 'cypress', 'jest', 'mocha', 'junit', 'testng',
        'pytest', 'postman', 'soapui', 'jmeter', 'loadrunner',
        'appium', 'cucumber', 'specflow', 'robot framework'
    ],

    # --- Business Systems ---
    'Business Systems': [
        'sap', 'salesforce', 'microsoft dynamics', 'oracle erp',
        'netsuite', 'workday', 'peoplesoft', 'sage', 'odoo', 'zoho',
        'hubspot', 'microsoft power platform', 'sharepoint'
    ],

    # --- IT & Administration ---
    'IT & Admin': [
        'active directory', 'windows server', 'linux', 'ubuntu',
        'centos', 'red hat', 'macos', 'vmware', 'hyper-v', 'citrix',
        'azure ad', 'office 365', 'exchange', 'powershell'
    ],

    # --- Business & Management ---
    'Business & Management': [
        'control models', 'business relationships', 'financial modeling',
        'risk management', 'strategic planning', 'project management'
    ],

    # --- Design & Creative ---
    'Design & Creative': [
        'figma', 'adobe xd', 'sketch', 'illustrator', 'photoshop',
        'indesign', 'after effects', 'premiere pro', 'maya', 'blender',
        'autocad', 'solidworks', 'revit', '3d modeling'
    ],

    # --- Industry-Specific ---
    'Healthcare IT': [
        'hl7', 'fhir', 'epic', 'cerner', 'meditech', 'hipaa compliance',
        'electronic health records'
    ],
    'Manufacturing': [
        'plc', 'scada', 'cnc', 'cad/cam', 'industry 4.0', 'iot',
        'lean manufacturing', 'six sigma'
    ],
    'Logistics': [
        'wms', 'tms', 'supply chain', 'inventory management',
        'sap mm', 'sap sd', 'sap ewm'
    ],
    'Marketing Tech': [
        'seo', 'sem', 'ppc', 'google analytics', 'google tag manager',
        'facebook ads', 'linkedin ads', 'marketing automation',
        'content marketing', 'social media marketing', 'email marketing'
    ],
    'Office Tools': [
        'word', 'ms office', 'outlook', 'ms office suite', 'powerpoint',
        'excel', 'google docs', 'google sheets'
    ]
}

# Flatten all keywords into a single set (for faster matching)
all_skills = set()
for category in SKILL_CATEGORIES.values():
    all_skills.update([skill.lower() for skill in category])

# Optionally: Add variations (e.g., "C++" -> "c++", "cplusplus")
all_skills.add("cplusplus")  # Example additional variation

import re

def extract_skills_simple(text):
    if not isinstance(text, str):
        return []
    
    text_lower = text.lower()
    found_skills = []
    
    for skill in all_skills:
        # Match whole words only (to avoid partial matches)
        if re.search(rf'\b{re.escape(skill)}\b', text_lower):
            found_skills.append(skill)
    
    return list(set(found_skills))  # Remove duplicates

In [98]:
mask = df.skills.apply(lambda x : len(x)) == 0
df.loc[mask , "skills"] = df[mask].requirements.apply(extract_skills_simple)

In [106]:
df[df["skills"].apply(lambda x : len(x)) == 0]

,title,company,location,job_type,employment_type,description,requirements,job_category,hard_skills,languages,experience,degree,soft_skills,skills
58,Consultant Artificial Intelligence,adesso SE,"Aachen, Bonn, Dortmund, Düsseldorf, Essen, Fra...",Feste Anstellung,"Homeoffice möglich, Vollzeit","With this guideline, we have been setting regi...",(Specialist) Abitur with good grades-especiall...,"IT|Data & AI|AI, LLM & NLP",{},"[german, english]",None,None,None,[]
74,Artificial Intelligence Specialist,SGS Holding Deutschland B.V. & Co. KG,Hamburg,Feste Anstellung,"Homeoffice möglich, Vollzeit",Life is Alway's About Becoming ... In life it ...,You have (soon) your Abitur and want to start ...,"IT|Data & AI|AI, LLM & NLP",{},[german],None,"{'bachelor's': 'degree', 'degree': 'degree', '...","[analytical, communication, initiative]",[]
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
106,Business Development Cpo Office,DFMG Deutsche Funkturm GmbH,Leverkusen,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit",VR Smart Finanz AG belongs to the DZ Bank Grou...,PhD in life or data sciences and/or MD in medi...,Business|Sales & Marketing|Sales,{},None,3 years,None,[leadership],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
3008,Employee It Development Leadership Skills,Regierung von Oberbayern,"München, Rosenheim, Ingolstadt",Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","We are a modern, powerful and future -oriented...",This should be the right thing for you to trai...,Public Sector & Non-Profit|Education & Research,{},None,None,None,[management],[]
3009,Employee It Development Support,Regierung von Oberbayern,"München, Rosenheim, Ingolstadt",Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","We are a modern, powerful and future -oriented...",This should be the right thing for you to trai...,IT|IT Support|Helpdesk & Support,{},None,None,None,None,[]
3018,Information Security Advisor,neoshare AG,"München, Frankfurt a. M.",Feste Anstellung,"Homeoffice möglich, Vollzeit","NEOSHARE AG, founded in Munich in 2019, quickl...",General/subject-bound high school graduation s...,Legal|Legal Practice,{},None,None,None,None,[]


In [111]:
mask = df.skills.apply(lambda x : len(x)) == 0
df.loc[mask , "skills"] = df[mask].title.apply(extract_skills_simple)

In [112]:
df[df["skills"].apply(lambda x : len(x)) == 0]

,title,company,location,job_type,employment_type,description,requirements,job_category,hard_skills,languages,experience,degree,soft_skills,skills
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
106,Business Development Cpo Office,DFMG Deutsche Funkturm GmbH,Leverkusen,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit",VR Smart Finanz AG belongs to the DZ Bank Grou...,PhD in life or data sciences and/or MD in medi...,Business|Sales & Marketing|Sales,{},None,3 years,None,[leadership],[]
113,Industrial Engineer Digital Transformation,WTSH GmbH,Kiel,Befristeter Vertrag,"Homeoffice möglich, Vollzeit",With us you have an impact on the development ...,You have a completed degree in (economic) comp...,Engineering|Industrial & Systems,{},"[german, english]",5 years of,"{'degree': 'degree', 'economic': 'field', 'eng...",[communication],[]
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
3005,Engineer City Planner Landscape Architect Obje...,Stadt Hameln,Hameln,Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","This is what the city of Hameln, also known as...",This position requires fluent German language ...,IT|IT Management|Enterprise Architecture,{},[german],5 years of,None,"[management, collaboration, communication]",[]
3007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},None,None,None,None,[]
3008,Employee It Development Leadership Skills,Regierung von Oberbayern,"München, Rosenheim, Ingolstadt",Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","We are a modern, powerful and future -oriented...",This should be the right thing for you to trai...,Public Sector & Non-Profit|Education & Research,{},None,None,None,[management],[]


In [114]:
df = df.dropna(subset=["title"])

In [116]:
df[df["skills"].apply(lambda x : len(x)) == 0]

,title,company,location,job_type,employment_type,description,requirements,job_category,hard_skills,languages,experience,degree,soft_skills,skills
106,Business Development Cpo Office,DFMG Deutsche Funkturm GmbH,Leverkusen,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit",VR Smart Finanz AG belongs to the DZ Bank Grou...,PhD in life or data sciences and/or MD in medi...,Business|Sales & Marketing|Sales,{},None,3 years,None,[leadership],[]
113,Industrial Engineer Digital Transformation,WTSH GmbH,Kiel,Befristeter Vertrag,"Homeoffice möglich, Vollzeit",With us you have an impact on the development ...,You have a completed degree in (economic) comp...,Engineering|Industrial & Systems,{},"[german, english]",5 years of,"{'degree': 'degree', 'economic': 'field', 'eng...",[communication],[]
133,Software Developer Genai Machine Learning,REWE digital,Köln,Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","TÜV Süd has stood for security, quality and su...",NaN,IT|Software Development|Full-Stack Development,{},None,None,None,None,[]
138,Computer Science Software Technology Training ...,GEA Farm Technologies GmbH,Bönen,"Ausbildung, Studium",Vollzeit,Why are you exactly right with us? Because we ...,General university entrance qualifications wit...,IT|Software Development|Full-Stack Development,{},"[german, english]",None,None,[initiative],[]
141,Training Computer Science Network Technology S...,GEA Farm Technologies GmbH,Bönen,"Ausbildung, Studium",Vollzeit,"As a worldwide company, we need an IT landscap...",General university entrance qualifications wit...,IT|Software Development|Full-Stack Development,{},"[german, english]",None,None,[initiative],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,Merchant Forwarding Logistics Service,Kiessling Spedition,Regenstauf,"Ausbildung, Studium",Teilzeit,The professional design of logistical processe...,• A preference for technical challenges- also ...,Business|Procurement|Supply Chain,{},"[german, english]",None,None,"[communication, open-minded]",[]
2992,Specialist Warehouse Logistics,Kiessling Spedition,Regenstauf,"Ausbildung, Studium",Teilzeit,The professional design of logistical processe...,"RequirementsAt Code17, we value passion, creat...",Business|Procurement|Supply Chain,{},"[german, english]",5 years of,None,"[collaboration, creativity]",[]
3005,Engineer City Planner Landscape Architect Obje...,Stadt Hameln,Hameln,Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","This is what the city of Hameln, also known as...",This position requires fluent German language ...,IT|IT Management|Enterprise Architecture,{},[german],5 years of,None,"[management, collaboration, communication]",[]
3008,Employee It Development Leadership Skills,Regierung von Oberbayern,"München, Rosenheim, Ingolstadt",Feste Anstellung,"Homeoffice möglich, Teilzeit, Vollzeit","We are a modern, powerful and future -oriented...",This should be the right thing for you to trai...,Public Sector & Non-Profit|Education & Research,{},None,None,None,[management],[]


## we can't do anything about the rest 719 rows they look good although they don't have requirements

#### now we deal with the degree and make other columns as dict or list types

In [136]:
df[df.degree != None]

,title,company,location,job_type,employment_type,description,requirements,job_category,hard_skills,languages,experience,degree,soft_skills,skills
0,Procurement Specialist Data Science Artificial...,Leadec Management Central Europe BV & Co. KG,Stuttgart,Feste Anstellung,"Homeoffice möglich, Vollzeit","With 14,500 employees worldwide and more than ...",You bring many years of professional experienc...,Business|Procurement|Strategic Sourcing,"{'sql': 'databases', 'python': 'programming la...","[german, english]",None,"{'degree': 'degree', 'economic': 'field', 'eng...",[management],"[sql, python, programming language, data scien..."
1,Dhbw Studies - Data Science Artificial Intelli...,Liebherr-International Deutschland GmbH,Biberach,NaN,NaN,Monari is a future-oriented and international ...,Successfully completed degree in business info...,IT|Data & AI|Data Science,"{'python': 'programming languages', 'databrick...",[english],None,"{'degree': 'degree', 'business': 'field'}",[communication],"[python, databricks, spark, cloud computing, c..."
2,Data Science Area Python Databricks,ARAG SE,Düsseldorf,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit",IONITY is a joint venture of the car manufactu...,You have a successfully completed degree in th...,IT|Data & AI|Data Science,"{'programming language': 'other', 'data scienc...",[german],None,"{'degree': 'degree', 'computer science': 'field'}","[analytical, communication, initiative]","[programming language, data science]"
3,Dual Studies Data Science Artificial Intelligence,Deloitte,"Düsseldorf, Mannheim, Stuttgart","Ausbildung, Studium",Vollzeit,ZS2Radätteilen GmbH is one of the leading Germ...,Completed scientific university studies in com...,IT|Data & AI|Data Science,"{'devops/mlops': 'other', 'data science': 'dat...","[german, english]",None,None,[communication],"[devops/mlops, data science]"
4,Dual Studies Data Science Artificial Intellige...,Lidl Stiftung & Co. KG,Neckarsulm,"Ausbildung, Studium",Vollzeit,Everything started with an adhesive film. Toda...,A completed course of study in a relevant area...,IT|Data & AI|Data Science,{'data science': 'data science & ai'},None,None,None,[analytical],[data science]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,Operations Manager Customer Service C24 Bank,CHECK24,Frankfurt am Main,Feste Anstellung,Vollzeit,C24 Bank GmbH is part of the Check24 Group. Un...,Completed studies (diploma [FH]/Bachelor) in t...,IT|IT Management|Product & Project Management,"{'java': 'programming languages', 'python': 'p...",None,None,"{'bachelor': 'degree', 'diploma': 'degree', 'd...","[analytical, communication]","[java, python, sql, css, javascript, programmi..."
3021,Cloud Architekt,ZDF,Mainz,Feste Anstellung,"Teilzeit, Vollzeit",The Planning and Realization business area (PU...,You have a completed training as a bank clerk ...,IT|DevOps & Cloud|Cloud Engineering,{},None,None,None,[communication],[cloud]
3022,It Service Engineer-Analyst Smart Meter Softwa...,Schleupen SE,Ettlingen,Feste Anstellung,"Homeoffice möglich, Vollzeit",We are an innovative medium -sized software co...,Completed vocational training as a specialist ...,Engineering|Electrical Engineering,{'ms office': 'office tools'},None,None,None,"[management, communication]",[ms office]
3023,It Specialist,FERCHAU – Connecting People and Technologies,Norderstedt,Feste Anstellung,"Homeoffice möglich, Vollzeit","Combining people and technologies, designing t...",Completed university education at an applied u...,Customer Service|Client Relations,"{'outlook': 'office tools', 'excel': 'bi & ana...",[german],None,None,"[management, initiative]","[outlook, excel, word]"


In [149]:
list(x for x in df.degree[3020].keys() if df.degree[3020].get(x) == 'degree')

['bachelor', 'diploma', 'degree']

In [168]:
df.loc[df.degree.notnull(), "degree_type"] = \
df[df.degree.notnull()].degree.apply(lambda x : list(y for y in x.keys() if x.get(y) == "degree"))

In [172]:
df.loc[df.degree.notnull(), "degree_field"] = \
df[df.degree.notnull()].degree.apply(lambda x : list(y for y in x.keys() if x.get(y) == "field"))

## now let's deal with the job_category for last

In [179]:
re.search(r"^([^|]+)", df.job_category[0]).group(1)

'Business'

In [209]:
df.loc[:,"sub_category_1"] = df.job_category.apply(lambda x : re.search(r"^([^|]+)",x).group(1))

In [210]:
df.loc[:,"sub_category_2"] = df.job_category.apply(lambda x : re.search(r"^[^|]+(?:\|([^|]+))?", x).group(1))
# re.search(r"([^|]+)$", df.job_category[380]).group(1)
# df.job_category[2980]

In [213]:
df.loc[:, "sub_category_3"] = df.job_category.apply(lambda x : re.search(r"([^|]+)$",x).group(1))

# last cleaning of the data now 

In [218]:
df = df.drop(columns=["description", "requirements", "job_category", "hard_skills", "degree"])

In [219]:
df

,title,company,location,job_type,employment_type,languages,experience,soft_skills,skills,degree_type,degree_field,sub_category_1,sub_category_2,sub_category_3
0,Procurement Specialist Data Science Artificial...,Leadec Management Central Europe BV & Co. KG,Stuttgart,Feste Anstellung,"Homeoffice möglich, Vollzeit","[german, english]",None,[management],"[sql, python, programming language, data scien...",[degree],"[economic, engineering, computer science, busi...",Business,Procurement,Strategic Sourcing
1,Dhbw Studies - Data Science Artificial Intelli...,Liebherr-International Deutschland GmbH,Biberach,NaN,NaN,[english],None,[communication],"[python, databricks, spark, cloud computing, c...",[degree],[business],IT,Data & AI,Data Science
2,Data Science Area Python Databricks,ARAG SE,Düsseldorf,"Studentenjobs, Werkstudent","Homeoffice möglich, Teilzeit",[german],None,"[analytical, communication, initiative]","[programming language, data science]",[degree],[computer science],IT,Data & AI,Data Science
3,Dual Studies Data Science Artificial Intelligence,Deloitte,"Düsseldorf, Mannheim, Stuttgart","Ausbildung, Studium",Vollzeit,"[german, english]",None,[communication],"[devops/mlops, data science]",NaN,NaN,IT,Data & AI,Data Science
4,Dual Studies Data Science Artificial Intellige...,Lidl Stiftung & Co. KG,Neckarsulm,"Ausbildung, Studium",Vollzeit,None,None,[analytical],[data science],NaN,NaN,IT,Data & AI,Data Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,Operations Manager Customer Service C24 Bank,CHECK24,Frankfurt am Main,Feste Anstellung,Vollzeit,None,None,"[analytical, communication]","[java, python, sql, css, javascript, programmi...","[bachelor, diploma, degree]","[technical, computer science]",IT,IT Management,Product & Project Management
3021,Cloud Architekt,ZDF,Mainz,Feste Anstellung,"Teilzeit, Vollzeit",None,None,[communication],[cloud],NaN,NaN,IT,DevOps & Cloud,Cloud Engineering
3022,It Service Engineer-Analyst Smart Meter Softwa...,Schleupen SE,Ettlingen,Feste Anstellung,"Homeoffice möglich, Vollzeit",None,None,"[management, communication]",[ms office],NaN,NaN,Engineering,Electrical Engineering,Electrical Engineering
3023,It Specialist,FERCHAU – Connecting People and Technologies,Norderstedt,Feste Anstellung,"Homeoffice möglich, Vollzeit",[german],None,"[management, initiative]","[outlook, excel, word]",NaN,NaN,Customer Service,Client Relations,Client Relations


In [226]:
pd.set_option('display.max_rows', 150)
df.experience.value_counts()

experience
5 years of                 46
2 years of                 41
3 years of                 41
3 years of experience      37
5 years                    34
5+ years of experience     28
3 years                    25
5 years of experience      25
2 years of experience      23
3+ years of experience     22
6 months                   14
3+ years of                14
5+ years of                14
10 years of experience     13
2 years                    13
2+ years                   12
10 years of                12
10+ years                  11
4 years of                 11
1 year                      8
5+ years                    7
10+ years of experience     7
2+ years of                 6
8 years of                  6
3+ years                    6
4 years of experience       6
2+ years of experience      5
5 years experience          5
4+ years of                 5
10+ years of                5
7 years of                  5
4 years                     5
6+ years of experience      4

In [227]:
def clean_experience(text):
    if not isinstance(text, str) or not text.strip():
        return None
    
    # Match numbers with optional units (years/months)
    match = re.search(
        r"(\d+)\+?\s*(?:years?|yrs?|months?|mos?|y|m)?\b", 
        text, 
        flags=re.IGNORECASE
    )
    if not match:
        return None
    
    num = match.group(1)  # Extract number (e.g., "5")
    
    # Determine unit (default to years if ambiguous)
    if re.search(r"months?|mos?|m\b", text, re.IGNORECASE):
        unit = "m"
    else:
        unit = "y"  # Default to years
    
    return f"{num}{unit}"

In [228]:
df["experience_periode"] = df.experience.apply(clean_experience)

In [232]:
df= df.drop(columns=["experience"],axis=1)

In [234]:
df.job_type.value_counts()

job_type
Feste Anstellung                                                                      2326
Studentenjobs, Werkstudent                                                             168
Ausbildung, Studium                                                                    142
Befristeter Vertrag                                                                     96
Praktikum                                                                               72
Berufseinstieg/Trainee                                                                  16
Berufseinstieg/Trainee, Feste Anstellung                                                14
Praktikum, Studentenjobs, Werkstudent                                                   11
Promotion/Habilitation                                                                   9
Freie Mitarbeit/Projektmitarbeit                                                         8
Bachelor-/Master-/Diplom-Arbeiten                                                

In [235]:
df.employment_type.value_counts()

employment_type
Homeoffice möglich, Vollzeit              1349
Vollzeit                                  1018
Homeoffice möglich, Teilzeit, Vollzeit     203
Homeoffice möglich, Teilzeit               118
Teilzeit                                   117
Teilzeit, Vollzeit                          75
Homeoffice möglich                           1
Name: count, dtype: int64

In [242]:
df.loc[:,"Homeoffice"] = df.employment_type.str.contains("Homeoffice",case= False, na= False)

In [244]:
df.loc[:,"teilzeit"] = df.employment_type.str.contains("Teilzeit",case = False, na= False)
df.loc[:,"Vollzeit"] = df.employment_type.str.contains("Vollzeit", case = False, na=False)

In [247]:
df = df.drop(columns=["employment_type"],axis=1)

In [267]:
df.job_type.value_counts()

job_type
Feste Anstellung                                                                      2326
Studentenjobs, Werkstudent                                                             168
Ausbildung, Studium                                                                    142
Befristeter Vertrag                                                                     96
Praktikum                                                                               72
Berufseinstieg/Trainee                                                                  16
Berufseinstieg/Trainee, Feste Anstellung                                                14
Praktikum, Studentenjobs, Werkstudent                                                   11
Promotion/Habilitation                                                                   9
Freie Mitarbeit/Projektmitarbeit                                                         8
Bachelor-/Master-/Diplom-Arbeiten                                                

In [274]:
df.loc[df.job_type.notna(),"job_type"] = df[df.job_type != None].job_type.apply(lambda x : list(item.strip() for item in str(x).split(",")))

In [279]:
df.loc[df.sub_category_2.isna(),"sub_category_2"]= df[df.sub_category_2.isna()].sub_category_3

In [280]:
df.isna().sum()

title                    0
company                  0
location                 2
job_type                21
languages             1185
soft_skills            834
skills                   0
degree_type           1785
degree_field          1785
sub_category_1           0
sub_category_2           0
sub_category_3           0
experience_periode    2242
Homeoffice               0
teilzeit                 0
Vollzeit                 0
dtype: int64

In [281]:
df.to_csv("clean_data.csv")